### SODA for SPARC reports 2022 Update

In [ ]:
!pip install oauth2client
!pip install google-api-python-client
!pip install ipywidgets
!pip install plotly
!pip install tqdm -U

### Pre-requisite Installations if Needed

In [1]:
import argparse
import pandas as pd
import json
import os
import ipywidgets as widgets
from pathlib import Path
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import calendar

import uuid

from tqdm.notebook import tqdm, trange

import plotly
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio

from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools
from helper_functions import initialize_analyticsreporting, get_report, print_response, VIEW_ID, next_date_interval, progress_bar_counter

pio.renderers.default = "iframe"

analytics = initialize_analyticsreporting()

### --------------------------------------------------------------------------------------------------------------------------------------------

#### The cell below will render the widgets needed to select the items in the graph. This cell only needs to be run ONCE (to show the widgets only). After display, you don't have to run this cell. The report/graph will include the end date.

### --------------------------------------------------------------------------------------------------------------------------------------------